#### This is a new version of the original p-adic embedding, with a few modifications:

- children of a node are now just the ones less than that node (and greater than the previous at the parent's level)
- we allow any integer base, not just primes (we don't need this to be a ring for our purposes)

That is, a positive integer $0 < x < 1$ is said to be $n$-adic (where $n \in \mathbb{Z}_{>1}$) iff, $\exists a,d \in \mathbb{Z}$ with $d > 0$ and $0 < a < n^d - 1$ such that

$$
x = \frac{a}{n^d}
$$

We can call the set of all $n$-adic numbers $\mathbb{Q}_n$

The children of $x = \frac{a}{n^d} \in \mathbb{Q}_n$ are described using the function $h: \mathbb{Q}_n \rightarrowtail \mathbb{Q}_n^{n-1}$ which is

$$
h(x) := \left\{\frac{b}{n^{d+1}} \mid n(a-1) < b < na \right\}
$$

Because we know the function is from $\mathbb{Q}_n$ onto $\mathbb{Q}_n^{n-1}$ (which is because $na - n(a-1) - 2 = n-1$), we know this generates an $(n-1)-ary$ tree.

Based on the previous, if we have $x,y \in \mathbb{Q}_n$ with $x = \frac{a}{n^d}$ and $y = \frac{b}{n^e}$ then $y$ is a descendant of $x$ iff $e > d$ and

$$
n^{e-d}(a-1) < b < n^{e-d}\cdot a
$$

Note that, although the quantity $n^{e-d}$ may be rather large, it is on the same order as $b$, as it's essentially just the scaling factor for moving $x$ down to $y$'s layer in the tree.

Additionally, $x$ is always an ancestor and a descendant of itself.

In [54]:
from typing import Union

In [72]:
class n_adic:
    
    '''
    d = None is taken to mean d = -inf (denom = 0 or x = inf)
    '''
    def __init__(self,a:int,n:int,d,reduce=True):
        assert((d is None) or (type(d) == int))
        if reduce:
            while (a % n) == 0:
                a //= n
                d -= 1
        self.a = a
        self.n = n
        self.d = d
        
        if d is None:
            self.par = []
            self.chld = []
            self.truedenom = 0
        else:
            self.par = None
            self.chld = None
            self.truedenom = None
    
    def __add__(self,other):
        if self.d > other.d:
            oscale = self.n**(self.d - other.d)
            sscale = 1
        elif other.d > self.d:
            oscale = 1
            sscale = self.n**(other.d - self.d)
        else:
            oscale = sscale = 1
        return n_adic(self.a*sscale + other.a*oscale,self.n,max(self.d,other.d),reduce=True)
    
    def __neg__(self):
        return n_adic(-self.a,self.n,self.d,reduce=False)
    
    def __sub__(self,other):
        return self + (-other)
    
    def __eq__(self,other):
        return (self.n == other.n) and (self.a == other.a) and (self.d == other.d)
    
    def __abs__(self):
        return n_adic(abs(self.a),self.n,self.d,reduce=False)
    
    def children(self):
        if self.chld is not None:
            return self.chld.copy()
        chdenom = self.d + 1
        self.chld = []
        for chnum in range(self.n*(self.a - 1) + 1,self.n*self.a):
            self.chld.append(n_adic(chnum,self.n,chdenom,reduce=False))
        return self.chld.copy()
    
    def is_ancestor_of(self,other):
        if self == other:
            return True
        if self.d >= other.d:
            return False
        #just need to check the main condition now
        scale = self.n**(other.d - self.d)
        rbound = scale*self.a
        if other.a >= rbound:
            return False
        lbound = rbound - scale
        return other.a >= lbound
    
    '''
    this version is ancestor-weighted ONLY (i.e. not descendant-weighted)
    '''
    def dist_to(self,other):
        raw_dist = abs(self-other)
        if self.is_ancestor_of(other):
            return raw_dist
        else:
            return raw_dist + n_adic(1,self.n,0,reduce=False)
    
    '''
    this is the descendent-ancestor weighted metric
    '''
    def dist_to_DA(self,other):
        if self == other:#both descendant and ancestor case
            return -n_adic(1,self.n,None,reduce=False)#take this to mean (negative) infinity
        s_anc_o = self.is_ancestor_of(other)
        o_anc_s = other.is_ancestor_of(self)
        #we know that not both of these ^^ are true at this point
        if s_anc_o:
            return self.d - other.d#-(e-d)
        elif o_anc_s:
            return other.d - self.d#-(d-e)
        else:
            raw_dist = abs(self-other)
            return raw_dist
            
    def __repr__(self):
        if self.truedenom is None:
            self.truedenom = self.n**self.d
        if self.truedenom == 0:
            return ("-" if self.a < 0 else "") + "INF"
        return "{}/{}".format(self.a,self.truedenom)

def lnat_inner(x,max_depth,depth=0):
    s = ''
    if depth > max_depth:
        return s
    
    s += '\t'*depth
    s += '[.' + str(x)
    s += '\n'
    
    #print all children
    for ch in x.children():
        s += lnat_inner(ch,max_depth,depth=depth+1)
    
    s += '\t'*depth
    s += ']\n'
    return s
    
def latex_n_adic_tree(n,depth):
    s = '\\Tree'
    x = n_adic(1,n,0)
    s += lnat_inner(x,depth)
    print(s)

In [70]:
n_adic(5,6,3).dist_to_DA(n_adic(1,6,1))

-2

In [74]:
n_adic(1,6,1).dist_to_DA(n_adic(5,6,3))

-2

In [73]:
n_adic(5,6,3).dist_to_DA(n_adic(5,6,3))

-INF

In [75]:
n_adic(16,6,3).dist_to_DA(n_adic(5,6,3))

11/216

In [50]:
latex_n_adic_tree(6,3)

\Tree[.1/1
	[.1/6
		[.1/36
			[.1/216
			]
			[.2/216
			]
			[.3/216
			]
			[.4/216
			]
			[.5/216
			]
		]
		[.2/36
			[.7/216
			]
			[.8/216
			]
			[.9/216
			]
			[.10/216
			]
			[.11/216
			]
		]
		[.3/36
			[.13/216
			]
			[.14/216
			]
			[.15/216
			]
			[.16/216
			]
			[.17/216
			]
		]
		[.4/36
			[.19/216
			]
			[.20/216
			]
			[.21/216
			]
			[.22/216
			]
			[.23/216
			]
		]
		[.5/36
			[.25/216
			]
			[.26/216
			]
			[.27/216
			]
			[.28/216
			]
			[.29/216
			]
		]
	]
	[.2/6
		[.7/36
			[.37/216
			]
			[.38/216
			]
			[.39/216
			]
			[.40/216
			]
			[.41/216
			]
		]
		[.8/36
			[.43/216
			]
			[.44/216
			]
			[.45/216
			]
			[.46/216
			]
			[.47/216
			]
		]
		[.9/36
			[.49/216
			]
			[.50/216
			]
			[.51/216
			]
			[.52/216
			]
			[.53/216
			]
		]
		[.10/36
			[.55/216
			]
			[.56/216
			]
			[.57/216
			]
			[.58/216
			]
			[.59/216
			]
		]
		[.11/36
			[.61/216
			]
			[.62/216
			]
			[.63/216
			]
			[.64/216
			]
			[.65/216
			]
		

In [52]:
n_adic(3,6,1).dist_to(n_adic(19,6,2))

37/36

In [35]:
n_adic(3,6,1).is_ancestor_of(n_adic(18,6,2))

True

In [34]:
n_adic(3,6,1).children()

[13/36, 14/36, 15/36, 16/36, 17/36]